In [1]:
#libraries
import ipywidgets as widgets
from ipywidgets import Layout, HBox, Label
from IPython.display import display, clear_output
import pandas as pd
from s3fs.core import S3FileSystem
import numpy as np
import shapefile
from shapely.geometry.polygon import LinearRing
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib.mlab import griddata

import codecs
import json

%matplotlib inline

#functions
from s3_functions import *
from shp_functions import *

In [2]:
s3 = S3FileSystem(anon=True)
bucket = 'ptrac-copano'
extent = (-97.3, -96.8, 27.9, 28.3)
zone_number = 14
zone_letter = 'R'
month_dict = {
    'April': '0401',
    'May': '0501',
    'June': '0601',
    'July': '0701',
    'August': '0801',
}
grid_resolution = 500

## Define the on_click function here

In [3]:
def read_data(year, month, coords):    
    #create the keys for the lat/lon files, this is their path in S3
    lat_key = 'data/{}/{}/lat.csv'.format(year, month)
    lon_key = 'data/{}/{}/lon.csv'.format(year, month)
    
    #pandas 0.20 uses s3fs to access S3 objects now (previous versions used boto or boto3)
    print('Reading latitude')
    lat = pd.read_csv(s3.open('{}/{}'.format(bucket, lat_key), mode='rb'), parse_dates=True, index_col=0)
    print('Reading longitude')
    lon = pd.read_csv(s3.open('{}/{}'.format(bucket, lon_key), mode='rb'), parse_dates=True, index_col=0)
    print('Reading velocity')
    outflw1 = read_outflw1(year, month, s3, bucket)
    print('Reading salinity')
    avesal = read_avesalD(year, month, s3, bucket)
    print('Done!')
    
    cn = [int(x) for x in list(outflw1.keys())]
    cknodes = coords.loc[cn]
    
    return lat, lon, outflw1, avesal, cknodes

## Plotting functions here

In [52]:
def init_plot(extent, geoms):
    #ring here is the box where the particles are dropped
    ring_lats = [28.142815, 28.142622, 28.128572, 28.128588]
    ring_lons = [-97.057931, -97.041549, -97.041542, -97.057923]
    ring = LinearRing(list(zip(ring_lons, ring_lats)))
    #create the figure
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection=ccrs.Miller())
    ax.set_extent(extent, ccrs.Miller())
    #this adds the shapefile
    ax.add_geometries(geoms, ccrs.Miller(), facecolor='none', edgecolor='black')
    #this adds the ring
    ax.add_geometries([ring], ccrs.Miller(), facecolor='none', edgecolor='red')
    return ax
    
def first_frame(ax, lat, lon, outflw1, cknodes, coords_clipped, grid_resolution, avesal, mask):
    index = lon.index[0]
    time_text = ax.text(0.02, 0.93, index, bbox=dict(facecolor='lightgray', alpha=1), 
                        fontsize=14, weight='medium', transform=ax.transAxes)
    #sctr = ax.scatter(lon.iloc[0], lat.iloc[0])
    sctr = ax.scatter(lon.iloc[100], lat.iloc[100], marker='.', facecolor='lime', edgecolor='k', linewidth=0.1, s=25)
    u = [outflw1[k]['u'][0] for k in outflw1]
    v = [outflw1[k]['v'][0] for k in outflw1]
    quiv = ax.quiver(cknodes['lon'], cknodes['lat'], u, v, scale=8, facecolor='white',
                     edgecolors='black', linewidth=0.3)

    lati = np.linspace(coords_clipped['lat'].min(), coords_clipped['lat'].max(), grid_resolution)
    loni = np.linspace(coords_clipped['lon'].min(), coords_clipped['lon'].max(), grid_resolution)

    sal = np.array(avesal[coords_clipped.index].iloc[0])
    sali = griddata(coords_clipped['lon'], coords_clipped['lat'], sal, loni, lati, interp='linear')
    sali.mask = ~mask
    levels = np.arange(0, 30.5, 0.5)
    ticks = np.arange(0, 30.5, 2)
    contf = ax.contourf(loni, lati, sali, zorder=0, cmap=plt.cm.seismic, vmin=0, vmax=30, 
                        levels=levels, extend='max', alpha=0.7)
    return time_text, sctr, quiv, lati, loni, levels, ticks, contf

def update_plot(time_text, new_text):
    time_text.set_text(new_text)

## Try creating a class to handle widget on_click

In [53]:
class widget_handler():
    def __init__(self):
        self.year_dropdown = widgets.Dropdown(
            options = ['1993', '1995', '1997', '2007', '2009', '2011'],
            value = '1993',
            description = 'Year:',
        )
        
        self.month_dropdown = widgets.Dropdown(
            #options = ['April', 'May', 'June', 'July', 'August'],
            options = ['0401', '0501', '0601', '0701', '0801'],
            #value = 'April',
            value = '0401',
            description = 'Month:',
        )
        
        self.get_data = widgets.Button(
            description = 'Get Data',
            button_style = 'info',
            tooltip = 'Click here to start downloading the appropriate data',
            icon = '',
        )
        
        self.make_plot = widgets.Button(
            description = 'Initialize Plot',
            button_style = '',
            tooltip = 'Click here to initialize the plot with appropriate data',
            icon = '',
            disabled = True,
        )        
        
        #add print statements to these so it doesn't look stalled
        print('Reading coordinates')
        self.coords, self.coords_clipped, self.loni, self.lati = read_coords(zone_number, zone_letter,
                                                                             extent, grid_resolution,
                                                                             s3, bucket)
        print('Reading shapefile')
        self.r, self.polygon, self.geoms = read_shapefile(s3, bucket)
        print('Reading mask')
        self.mask = read_mask(s3, bucket)
        
        clear_output()
        
        self.get_data.on_click(self.get_data_click)
        self.make_plot.on_click(self.make_plot_click)
        
        display(self.year_dropdown, self.month_dropdown, self.get_data, self.make_plot)
        
    def get_data_click(self, b):
        self.year = self.year_dropdown.get_interact_value()
        self.month = self.month_dropdown.get_interact_value()
        clear_output()
        self.lat, self.lon, self.outflw1, self.avesal, self.cknodes = read_data(self.year, self.month, self.coords)
        self.get_data.set_trait('icon', 'check')
        self.get_data.set_trait('button_style', 'success')
        self.make_plot.set_trait('disabled', False)
        self.make_plot.set_trait('button_style', 'info')
        
    def make_plot_click(self, b):
        clear_output()
        self.ax = init_plot(extent, self.geoms)
        self.time_text, self.sctr, self.quiv, self.lati, self.loni, self.levels, self.ticks, self.contf = \
        first_frame(self.ax, self.lat, self.lon, self.outflw1, self.cknodes, self.coords_clipped, \
                    grid_resolution, self.avesal, self.mask)
        #This works -> self.time_text.set_text('changed')
        self.make_plot.set_trait('icon', 'check')
        self.make_plot.set_trait('button_style', 'success')
        self.date_slider = widgets.SelectionSlider(
            options=self.lon.index.tolist(), 
            description='Dates',
            layout=Layout(width='75%', margin='20px'),
        )
        display(HBox([self.date_slider]))
        self.date_slider.observe(self.slider_change)
        
    def slider_change(self, b):
        update_plot(self.time_text, 'new text')
        

In [54]:
f = widget_handler()

Reading latitude
Reading longitude


In [40]:
f.time_text.set_text('balls')

In [19]:
date_slider = widgets.SelectionSlider(options=f.lon.index.tolist(), 
                                           description='Dates',
                                           layout=Layout(width='75%', margin='20px'))
display(HBox([date_slider]))

date_slider.observe(update_plot)

1993-04-01 00:00:00
1993-04-01 02:30:00
1993-04-01 02:30:00
1993-04-01 02:30:00
1993-04-01 15:00:00
1993-04-01 15:00:00
1993-04-01 15:00:00
1993-04-01 22:30:00
1993-04-01 22:30:00
1993-04-01 22:30:00
1993-04-02 04:30:00
1993-04-02 04:30:00
1993-04-02 04:30:00
1993-04-02 09:30:00
1993-04-02 09:30:00
1993-04-02 09:30:00
1993-04-02 10:30:00
1993-04-02 10:30:00
1993-04-02 10:30:00
1993-04-02 13:00:00
1993-04-02 13:00:00
1993-04-02 13:00:00
1993-04-02 14:30:00
1993-04-02 14:30:00
1993-04-02 14:30:00
1993-04-02 15:30:00
1993-04-02 15:30:00
1993-04-02 15:30:00
1993-04-03 02:30:00
1993-04-03 02:30:00
1993-04-03 02:30:00
1993-04-03 11:00:00
1993-04-03 11:00:00
1993-04-03 11:00:00
1993-04-03 20:00:00
1993-04-03 20:00:00
1993-04-03 20:00:00
1993-04-03 23:30:00
1993-04-03 23:30:00
1993-04-03 23:30:00
1993-04-04 03:00:00
1993-04-04 03:00:00
1993-04-04 03:00:00
1993-04-04 14:30:00
1993-04-04 14:30:00
1993-04-04 14:30:00
1993-04-04 18:00:00
1993-04-04 18:00:00
1993-04-04 18:00:00
1993-04-05 01:30:00


In [51]:
f.time_text.set_text('new text')
f.time_text.get_text()

'new text'